# 01 — Construção do Metadata e Split por Paciente

Projeto: Classificação de Pneumonia em Raio-X

Estrutura atual do projeto:

xray-project/
├── data/
│   ├── train/
│   │   ├── NORMAL/
│   │   └── PNEUMONIA/
│   └── test_images/
└── notebooks/

Objetivos deste notebook:

1. Construir um arquivo `train_metadata.csv` contendo:
   - filename
   - path absoluto
   - label (0 = NORMAL, 1 = PNEUMONIA)
   - patient_id extraído do nome do arquivo

2. Realizar divisão treino/validação utilizando GroupShuffleSplit
   baseado em paciente (evitando vazamento).

3. Salvar os arquivos:
   - data/metadata/train_metadata.csv
   - data/metadata/train_split.csv
   - data/metadata/val_split.csv

Todos os caminhos serão definidos a partir do diretório raiz do projeto.


In [18]:
import os

# Caminho absoluto do diretório onde o notebook está
NOTEBOOK_DIR = os.getcwd()

# O root do projeto é o diretório pai de notebooks
PROJECT_ROOT = os.path.abspath(os.path.join(NOTEBOOK_DIR, ".."))

print("Notebook está em:", NOTEBOOK_DIR)
print("Project root detectado:", PROJECT_ROOT)


Notebook está em: c:\projects\xray-project\notebooks
Project root detectado: c:\projects\xray-project


## 1. Definição do Diretório Raiz do Projeto

Detectamos automaticamente o diretório raiz do projeto.
Isso evita problemas com caminhos relativos e impede a criação
de pastas erradas dentro de notebooks/.


In [19]:
train_root = os.path.join(PROJECT_ROOT, "data", "train")

print("Diretório de treino:", train_root)
print("Existe?", os.path.exists(train_root))


Diretório de treino: c:\projects\xray-project\data\train
Existe? True


In [20]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GroupShuffleSplit
import random
import torch

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

print("Semente fixada com sucesso.")


Semente fixada com sucesso.


## 2. Controle de Reprodutibilidade

Fixamos a semente global para garantir que:
- O split treino/validação será reproduzível.
- Experimentos futuros terão comportamento determinístico.


In [21]:
records = []

for label_name in ["NORMAL", "PNEUMONIA"]:
    class_dir = os.path.join(train_root, label_name)

    for filename in os.listdir(class_dir):
        if filename.endswith(".jpeg"):
            
            # Exemplo:
            # NORMAL-170867-0001.jpeg → patient_id = 170867
            patient_id = filename.split("-")[1]

            records.append({
                "filename": filename,
                "path": os.path.join(class_dir, filename),
                "label": 0 if label_name == "NORMAL" else 1,
                "patient_id": patient_id
            })

df = pd.DataFrame(records)

print("Total de imagens encontradas:", len(df))
df.head()


Total de imagens encontradas: 5232


,filename,path,label,patient_id
0,NORMAL-1003233-0001.jpeg,c:\projects\xray-project\data\train\NORMAL\NOR...,0,1003233
1,NORMAL-1012843-0001.jpeg,c:\projects\xray-project\data\train\NORMAL\NOR...,0,1012843
2,NORMAL-1014768-0001.jpeg,c:\projects\xray-project\data\train\NORMAL\NOR...,0,1014768
3,NORMAL-1023731-0001.jpeg,c:\projects\xray-project\data\train\NORMAL\NOR...,0,1023731
4,NORMAL-1029510-0001.jpeg,c:\projects\xray-project\data\train\NORMAL\NOR...,0,1029510


## 3. Construção do Metadata

Percorremos:

data/train/NORMAL
data/train/PNEUMONIA

Para cada imagem:
- Extraímos o patient_id do nome do arquivo.
- Atribuímos o rótulo correspondente.
- Salvamos o caminho absoluto da imagem.

O patient_id será usado para evitar vazamento entre treino e validação.


In [22]:
metadata_dir = os.path.join(PROJECT_ROOT, "data", "metadata")
os.makedirs(metadata_dir, exist_ok=True)

print("Diretório metadata:", metadata_dir)


Diretório metadata: c:\projects\xray-project\data\metadata


In [23]:
metadata_path = os.path.join(metadata_dir, "train_metadata.csv")
df.to_csv(metadata_path, index=False)

print("Arquivo salvo em:", metadata_path)


Arquivo salvo em: c:\projects\xray-project\data\metadata\train_metadata.csv


## 4. Divisão Treino/Validação por Paciente

Utilizamos GroupShuffleSplit para garantir que:

- Nenhum paciente apareça em treino e validação.
- A avaliação seja realista.
- Não haja vazamento de informação.


In [24]:
gss = GroupShuffleSplit(
    n_splits=1,
    test_size=0.2,
    random_state=42
)

for train_idx, val_idx in gss.split(
    df,
    df["label"],
    groups=df["patient_id"]
):
    train_df = df.iloc[train_idx].reset_index(drop=True)
    val_df = df.iloc[val_idx].reset_index(drop=True)


In [26]:
intersection = set(train_df["patient_id"]).intersection(set(val_df["patient_id"]))
print("Pacientes em comum:", len(intersection))
# Verificar vazamento


Pacientes em comum: 0


In [27]:
train_split_path = os.path.join(metadata_dir, "train_split.csv")
val_split_path = os.path.join(metadata_dir, "val_split.csv")

train_df.to_csv(train_split_path, index=False)
val_df.to_csv(val_split_path, index=False)

print("Train salvo em:", train_split_path)
print("Val salvo em:", val_split_path)


Train salvo em: c:\projects\xray-project\data\metadata\train_split.csv
Val salvo em: c:\projects\xray-project\data\metadata\val_split.csv
